In [1]:
import os
from dotenv import load_dotenv
from groq import Groq

import pandas as pd
from tqdm import tqdm
from sklearn.metrics import accuracy_score, classification_report
import numpy as np


In [2]:
load_dotenv()

GROQ_API_KEY = os.getenv("GROQ_API_KEY")
if GROQ_API_KEY is None:
    raise ValueError("❌ Missing GROQ_API_KEY in environment variables.")

client = Groq(api_key=GROQ_API_KEY)

def groq_generate(prompt, model):
    """Generic Groq requester with custom model."""
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are a tweet classifier that detects expressions of hope."},
            {"role": "user", "content": prompt},
        ],
        temperature=0.1,
        max_completion_tokens=80,
    )
    return response.choices[0].message.content.strip()


In [3]:
df = pd.read_csv("HSD.csv")
print("Dataset loaded:", len(df))
df.head()


Dataset loaded: 8256


,text,binary,multiclass
0,#USER# #USER# You can’t solve Nigerias problem...,Not Hope,Not Hope
1,#USER# #USER# #USER# So looking forward to to ...,Hope,Generalized Hope
2,#USER# This is the most runs they’ve scored al...,Not Hope,Not Hope
3,#USER# My point is: any vaccine batches that c...,Not Hope,Not Hope
4,Liberals were happy killing the businesses tha...,Not Hope,Not Hope


In [4]:
# ============================
# Subset BINARIO (Hope vs Not Hope)
# ============================
subset = df[df['binary'].isin(['Not Hope', 'Hope'])]

subsetBin = (
    subset
    .groupby('binary', group_keys=False)
    .apply(lambda x: x.sample(500, random_state=42))
    .sample(frac=1, random_state=42)
    .reset_index(drop=True)
)

# ============================
# Subset MULTICLASS
# ============================
subset2 = df[df['multiclass'].isin([
    'Generalized Hope',
    'Realistic Hope',
    'Unrealistic Hope'
])]

subsetMulti = (
    subset2
    .groupby('multiclass', group_keys=False)
    .apply(lambda x: x.sample(334, random_state=42))
    .sample(frac=1, random_state=42)
    .reset_index(drop=True)
)

print("Binary subset:", len(subsetBin))
print("Multiclass subset:", len(subsetMulti))


Binary subset: 1000
Multiclass subset: 1002


C:\Users\Theas\AppData\Local\Temp\ipykernel_36524\865934801.py:9: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.sample(500, random_state=42))
C:\Users\Theas\AppData\Local\Temp\ipykernel_36524\865934801.py:26: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.sample(334, random_state=42))


In [5]:
def clasificarBin_batch(lista_textos, prompt_version=1, df=None, model=None):
    """
    Clasifica una LISTA de textos en:
    - Hope
    - Not Hope

    prompt_version:
    1 → Zero-shot
    2 → One-shot (1 ejemplo)
    3 → Few-shot (hasta 5 ejemplos)

    Retorna SIEMPRE len(lista_textos) etiquetas.
    """

    VALID_LABELS = ["Hope", "Not Hope"]

    definicion_hope = "Hope means expressing optimism, trust, or desire for a better future."

    # ===================== BLOQUE NUMERADO =====================
    bloque_tweets = "\n".join(
        f"{i+1}. {texto}" for i, texto in enumerate(lista_textos)
    )

    # ===================== EJEMPLOS =====================
    ejemplo_texto = ""
    ejemplos_texto = ""

    if df is not None and "binary" in df.columns and not df.empty:

        # 🔹 ONE-SHOT → 1 ejemplo
        if prompt_version == 2:
            row = df.sample(1).iloc[0]
            ejemplo_texto = f'"{row["text"]}" → {row["binary"]}'

        # 🔹 FEW-SHOT → hasta 5 ejemplos
        elif prompt_version == 3:
            ejemplos = df.sample(min(5, len(df)))
            ejemplos_texto = "\n".join(
                f'"{row["text"]}" → {row["binary"]}'
                for _, row in ejemplos.iterrows()
            )

    # Fallback si no hay df
    if prompt_version == 2 and ejemplo_texto == "":
        ejemplo_texto = '"Things will get better soon." → Hope'

    if prompt_version == 3 and ejemplos_texto == "":
        ejemplos_texto = '"Things will get better soon." → Hope'

    # ===================== PROMPTS =====================

    # ---------- ZERO SHOT ----------
    if prompt_version == 1:
        prompt = f"""
Definition: {definicion_hope}

You will receive a numbered list of tweets.
Return EXACTLY ONE LABEL PER LINE, in the same order.
Valid labels:
- Hope
- Not Hope

Tweets:
{bloque_tweets}

Now output ONLY the labels, one per line, with no numbers and no extra text:
"""

    # ---------- ONE SHOT (1 ejemplo) ----------
    elif prompt_version == 2:
        prompt = f"""
Definition: {definicion_hope}

Example:
{ejemplo_texto}

You will receive a numbered list of tweets.
Return EXACTLY ONE LABEL PER LINE, in the same order.
Valid labels:
- Hope
- Not Hope

Tweets:
{bloque_tweets}

Now output ONLY the labels, one per line, with no numbers and no extra text:
"""

    # ---------- FEW SHOT (5 ejemplos) ----------
    else:
        prompt = f"""
Definition: {definicion_hope}

Examples:
{ejemplos_texto}

You will receive a numbered list of tweets.
Return EXACTLY ONE LABEL PER LINE, in the same order.
Valid labels:
- Hope
- Not Hope

Tweets:
{bloque_tweets}

Now output ONLY the labels, one per line, with no numbers and no extra text:
"""

    # ===================== CALL MODEL =====================
    raw = groq_generate(prompt, model=model)
    raw_lines = [l.strip() for l in raw.splitlines() if l.strip()]

    # ===================== PARSER ROBUSTO =====================
    labels = []

    for line in raw_lines:
        clean = line.strip()

        if clean == "":
            continue
        if clean.lower().startswith(("tweet", "label", "output")):
            continue
        if ":" in clean:
            clean = clean.split(":")[-1].strip()

        for v in VALID_LABELS:
            if v.lower() == clean.lower():
                labels.append(v)
                break

    # ===================== FALLBACK SEGURO =====================
    if len(labels) < len(lista_textos):
        labels += ["Not Hope"] * (len(lista_textos) - len(labels))
    elif len(labels) > len(lista_textos):
        labels = labels[:len(lista_textos)]

    return labels


In [8]:
def clasificarMulti_batch(lista_textos, prompt_version=1, df=None, model=None):
    """
    Classifies texts using PROMPT LABELS ONLY:
    - Generalized Hope
    - Realistic Hope
    - Fantasy Wish

    ❗ No dataset labels appear here.
    """

    PROMPT_LABELS = [
        "Generalized Hope",
        "Realistic Hope",
        "Fantasy Wish"
    ]

    # ---------- Dataset for shots (mapped) ----------
    df_hope = None
    if df is not None and "multiclass" in df.columns:
        df_hope = df.copy()
        df_hope["prompt_label"] = df_hope["multiclass"].map({
            "Generalized Hope": "Generalized Hope",
            "Abstract Hope": "Generalized Hope",   # replaced Abstract with Generalized
            "Realistic Hope": "Realistic Hope",
            "Unrealistic Hope": "Fantasy Wish"
        })
        df_hope = df_hope[df_hope["prompt_label"].notna()]

    bloque_tweets = "\n".join(f"{i+1}. {t}" for i, t in enumerate(lista_textos))

    RULES = """
CLASSIFICATION RULES:

FANTASY WISH:
- Impossible, miraculous, magical, or unrealistic outcomes.
- Fictional, cosmic, or hyperbolic wishes.

REALISTIC HOPE:
- Hope clearly tied to a specific, real-world situation or event.
- Clearly tied to a concrete context (event, date, person, institution).
- Plausible within normal reality.

GENERALIZED HOPE:
- Broad, non-specific expressions of hope.
- Blessings, prayers, encouragement, or vague optimism.
- NOT clearly tied to a specific event or situation.
"""

    # ---------- PROMPTS ----------
    if prompt_version == 1:  # Zero-shot
        prompt = f"""
Classify each tweet into ONE of:
- Generalized Hope
- Realistic Hope
- Fantasy Wish

{RULES}

Tweets:
{bloque_tweets}

Output ONLY one label per line:
"""

    elif prompt_version == 2:  # One-shot
        ex = df_hope.sample(1).iloc[0] if df_hope is not None else None
        ej_text = ex["text"] if ex is not None else "I hope a miracle fixes everything overnight."
        ej_label = ex["prompt_label"] if ex is not None else "Fantasy Wish"

        prompt = f"""
Classify each tweet into ONE of:
- Generalized Hope
- Realistic Hope
- Fantasy Wish

{RULES}

Example:
"{ej_text}" → {ej_label}

Tweets:
{bloque_tweets}

Output ONLY one label per line:
"""

    else:  # Few-shot
        ejemplos = []
        if df_hope is not None:
            for lbl in PROMPT_LABELS:
                sub = df_hope[df_hope["prompt_label"] == lbl]
                if not sub.empty:
                    r = sub.sample(1).iloc[0]
                    ejemplos.append(f'"{r["text"]}" → {lbl}')

        ejemplos_texto = "\n".join(ejemplos)

        prompt = f"""
Classify each tweet into ONE of:
- Generalized Hope
- Realistic Hope
- Fantasy Wish

{RULES}

Examples:
{ejemplos_texto}

Tweets:
{bloque_tweets}

Output ONLY one label per line:
"""

    raw = groq_generate(prompt, model=model)
    raw_lines = [l.strip() for l in raw.splitlines() if l.strip()]

    def normalize(label):
        t = label.lower()
        if "fantasy" in t:
            return "Fantasy Wish"
        if "realistic" in t:
            return "Realistic Hope"
        if "generalized" in t or "abstract" in t:
            return "Generalized Hope"  # unify Abstract → Generalized
        # fallback
        return "Generalized Hope"

    labels = [normalize(l) for l in raw_lines]

    if len(labels) < len(lista_textos):
        labels += ["Generalized Hope"] * (len(lista_textos) - len(labels))
    elif len(labels) > len(lista_textos):
        labels = labels[:len(lista_textos)]  # <-- fixed


    return labels


In [9]:
    MODELS = {
        # ---------- LLaMA family ----------
        "LLaMA-8B": "llama-3.1-8b-instant",
        "LLaMA-70B": "llama-3.3-70b-versatile",
    
        # ---------- LLaMA-4 (Instruct variants) ----------
        "Maverick-17B": "meta-llama/llama-4-maverick-17b-128e-instruct",
        "Scout-17B": "meta-llama/llama-4-scout-17b-16e-instruct",
    
        # ---------- External / Non-LLaMA ----------
        "Allam-7B": "allam-2-7b",
        "Kimi-K2": "moonshotai/kimi-k2-instruct",
        "Qwen-32B": "qwen/qwen3-32b"
    }


In [ ]:
from tqdm.auto import tqdm
from groq import RateLimitError
import pandas as pd

RESULTS = []

BATCH_SIZE = 20
NUM_RECORDS = 200

finBin = subsetBin.sample(NUM_RECORDS).copy()  # ✅ SOLO binary

# ⛔ MODELOS A SALTAR
SKIP_MODELS = []

print("⏭️ Skipping models:", SKIP_MODELS)

# ===================== LOOP PRINCIPAL =====================
for model_name, model_id in tqdm(
    MODELS.items(),
    desc="Modelos (Binary)",
    total=len(MODELS)
):

    if model_name in SKIP_MODELS:
        print(f"\n⏭️ SKIPPING MODEL (hardcoded): {model_name}\n")
        continue

    print(f"\n==========================")
    print(f"Testing MODEL (Binary): {model_name}")
    print("==========================\n")

    try:
        for prompt_version in tqdm(
            [1, 2, 3],
            desc=f"Prompts ({model_name})"
        ):

            col = f"{model_name}_p{prompt_version}"
            if col in finBin.columns:
                print(f"⏭️ {col} already exists, skipping")
                continue

            preds_bin = []
            batches_bin = range(0, len(finBin), BATCH_SIZE)

            for i in tqdm(
                batches_bin,
                desc=f"Binary p{prompt_version} — {model_name}",
                leave=False
            ):
                batch = finBin["text"].iloc[i:i+BATCH_SIZE].tolist()
                preds_bin.extend(
                    clasificarBin_batch(
                        batch,
                        prompt_version,
                        df=subsetBin,
                        model=model_id
                    )
                )

            finBin[col] = preds_bin
            print(f"{model_name} — Binary Prompt {prompt_version} DONE.")

        # 🔹 Guardado parcial por modelo
        finBin.to_csv(f"resultados_binary_{model_name}.csv", index=False)

    except Exception as e:
        print(f"\n⚠️ Error en modelo {model_name}: {e}")
        print("⏭️ Saltando este modelo y continuando.\n")
        continue

# ===================== GUARDADO FINAL =====================
finBin.to_csv("Btempraw, index=False)

print("\n✅ Evaluación BINARIA terminada. CSV guardado correctamente.")


In [10]:
from tqdm.auto import tqdm
from groq import RateLimitError
import pandas as pd

RESULTS = []

BATCH_SIZE = 20
NUM_RECORDS = 200

# Sample a subset of the dataset
finMulti = subsetMulti.sample(NUM_RECORDS).copy()

SKIP_MODELS = []
print("⏭️ Skipping models:", SKIP_MODELS)

for model_name, model_id in tqdm(MODELS.items(), desc="Modelos", total=len(MODELS)):

    if model_name in SKIP_MODELS:
        print(f"\n⏭️ SKIPPING MODEL (hardcoded): {model_name}\n")
        continue

    print(f"\n==========================")
    print(f"Testing MODEL: {model_name}")
    print("==========================\n")

    try:
        for prompt_version in tqdm([1, 2, 3], desc=f"Prompts ({model_name})"):

            col = f"{model_name}_p{prompt_version}"
            if col in finMulti.columns:
                print(f"⏭️ {col} already exists, skipping")
                continue

            preds_multi = []
            batches_multi = range(0, len(finMulti), BATCH_SIZE)

            for i in tqdm(
                batches_multi,
                desc=f"Multi p{prompt_version} — {model_name}",
                leave=False
            ):
                batch = finMulti["text"].iloc[i:i+BATCH_SIZE].tolist()
                batch_preds = clasificarMulti_batch(
                    batch,
                    prompt_version,
                    df=subsetMulti,
                    model=model_id
                )

                # Sanity check: all labels are valid
                for lbl in batch_preds:
                    if lbl not in ["Generalized Hope", "Realistic Hope", "Fantasy Wish"]:
                        print(f"⚠️ Unexpected label: {lbl}")
                        lbl = "Generalized Hope"

                preds_multi.extend(batch_preds)

            finMulti[col] = preds_multi
            print(f"{model_name} — Prompt {prompt_version} DONE.")

        # Save per model
        finMulti.to_csv(f"resultados_multiclass_{model_name}.csv", index=False)

    except Exception as e:
        print(f"\n⚠️ Error en modelo {model_name}: {e}")
        print("⏭️ Saltando este modelo y continuando.\n")
        continue

# Save final aggregated CSV
finMulti.to_csv("Mtempraw.csv", index=False)
print("\n✅ Evaluación terminada. CSV guardado correctamente.")


⏭️ Skipping models: []


Modelos:   0%|          | 0/7 [00:00<?, ?it/s]


Testing MODEL: LLaMA-8B



Prompts (LLaMA-8B):   0%|          | 0/3 [00:00<?, ?it/s]

Multi p1 — LLaMA-8B:   0%|          | 0/10 [00:00<?, ?it/s]

...

Qwen-32B — Prompt 1 DONE.


Multi p2 — Qwen-32B:   0%|          | 0/10 [00:00<?, ?it/s]

Qwen-32B — Prompt 2 DONE.


Multi p3 — Qwen-32B:   0%|          | 0/10 [00:00<?, ?it/s]

Qwen-32B — Prompt 3 DONE.

✅ Evaluación terminada. CSV guardado correctamente.


In [ ]:
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd
import os

print("===== 🧠 BINARY CLASSIFICATION (ALL MODELS) =====")

rows = []

def normalize_binary(label):
    if not isinstance(label, str):
        return "Not Hope"
    t = label.strip().lower()
    if "not hope" in t:
        return "Not Hope"
    if t == "hope":
        return "Hope"
    if "hope" in t and "not" not in t:
        return "Hope"
    return "Not Hope"

os.makedirs("used_prompts", exist_ok=True)

for model_name in MODELS.keys():
    print(f"\n======================")
    print(f"MODEL: {model_name}")
    print("======================")

    for p in [1, 2, 3]:
        col = f"{model_name}_p{p}"

        if col not in finBin.columns:
            print(f"⏭️ Missing {col}, skipping")
            continue

        finBin[f"{col}_norm"] = finBin[col].apply(normalize_binary)

        y_true = finBin["binary"]
        y_pred = finBin[f"{col}_norm"]

        acc = accuracy_score(y_true, y_pred)
        report = classification_report(
            y_true,
            y_pred,
            output_dict=True,
            zero_division=0
        )

        macro_f1 = report["macro avg"]["f1-score"]

        invalid_outputs = sum(
            1 for x in finBin[col]
            if not isinstance(x, str)
            or ("hope" not in x.lower())
        )

        print(f"\n--- Prompt {p} ---")
        print(f"Accuracy: {acc*100:.2f}%")
        print(f"Macro F1 (PRIMARY): {macro_f1:.4f}")
        print(classification_report(y_true, y_pred, zero_division=0))

        for label in ["Hope", "Not Hope"]:
            rows.append({
                "Model": model_name,
                "Prompt": p,
                "PromptType": ["zero-shot", "one-shot", "few-shot"][p-1],
                "Label": label,
                "Precision": report[label]["precision"],
                "Recall": report[label]["recall"],
                "F1": report[label]["f1-score"],
                "Support": report[label]["support"],
                "Accuracy": acc,
                "Primary_Macro_F1": macro_f1,
                "InvalidOutputs": invalid_outputs
            })

        rows.append({
            "Model": model_name,
            "Prompt": p,
            "PromptType": ["zero-shot", "one-shot", "few-shot"][p-1],
            "Label": "macro avg",
            "Precision": report["macro avg"]["precision"],
            "Recall": report["macro avg"]["recall"],
            "F1": macro_f1,
            "Support": report["macro avg"]["support"],
            "Accuracy": acc,
            "Primary_Macro_F1": macro_f1,
            "InvalidOutputs": invalid_outputs
        })

        rows.append({
            "Model": model_name,
            "Prompt": p,
            "PromptType": ["zero-shot", "one-shot", "few-shot"][p-1],
            "Label": "weighted avg",
            "Precision": report["weighted avg"]["precision"],
            "Recall": report["weighted avg"]["recall"],
            "F1": report["weighted avg"]["f1-score"],
            "Support": report["weighted avg"]["support"],
            "Accuracy": acc,
            "Primary_Macro_F1": macro_f1,
            "InvalidOutputs": invalid_outputs
        })

        with open(f"used_prompts/binary_{model_name}_prompt{p}.txt", "w", encoding="utf-8") as f:
            f.write(f"MODEL: {model_name}\nPROMPT VERSION: {p}\n")

df_binary_metrics = pd.DataFrame(rows)
df_binary_metrics.to_csv("Btemp.csv", index=False)

print("\n✅ Saved: Btemp.csv")


In [11]:
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd
import os

print("===== 🎨 MULTICLASS CLASSIFICATION (ALL MODELS) =====")

rows = []

# ============================
# Etiquetas válidas finales
# ============================
VALID_LABELS = [
    "Generalized Hope",
    "Realistic Hope",
    "Unrealistic Hope"
]

# ============================
# Normalización ROBUSTA
# ============================
def normalize_multiclass(label):
    if not isinstance(label, str):
        return "Generalized Hope"

    t = label.lower()
    if "fantasy" in t:
        return "Unrealistic Hope"
    if "realistic" in t:
        return "Realistic Hope"
    if "abstract" in t:
        return "Generalized Hope"

    return "Generalized Hope"



# ============================
# Guardar prompts usados
# ============================
os.makedirs("used_prompts", exist_ok=True)

# ============================
# LOOP POR MODELO Y PROMPT
# ============================
for model_name in MODELS.keys():
    print(f"\n======================")
    print(f"MODEL: {model_name}")
    print("======================")

    for p in [1, 2, 3]:
        col = f"{model_name}_p{p}"

        if col not in finMulti.columns:
            print(f"⏭️ Missing {col}, skipping")
            continue

        # Normalizar predicciones
        finMulti[f"{col}_norm"] = finMulti[col].apply(normalize_multiclass)

        y_true = finMulti["multiclass"]
        y_pred = finMulti[f"{col}_norm"]

        acc = accuracy_score(y_true, y_pred)
        report = classification_report(
            y_true,
            y_pred,
            output_dict=True,
            zero_division=0
        )

        macro_f1 = report["macro avg"]["f1-score"]

        # Conteo de outputs inválidos (antes de normalizar)
        invalid_outputs = sum(
            1 for x in finMulti[col]
            if not isinstance(x, str)
            or all(lbl.lower() not in x.lower() for lbl in ["general", "realistic", "fantasy", "unrealistic"])
        )

        print(f"\n--- Prompt {p} ---")
        print(f"Accuracy: {acc*100:.2f}%")
        print(f"Macro F1 (PRIMARY): {macro_f1:.4f}")
        print(classification_report(y_true, y_pred, zero_division=0))

        # ============================
        # Métricas por clase
        # ============================
        for label in VALID_LABELS:
            rows.append({
                "Model": model_name,
                "Prompt": p,
                "PromptType": ["zero-shot", "one-shot", "few-shot"][p-1],
                "Label": label,
                "Precision": report[label]["precision"],
                "Recall": report[label]["recall"],
                "F1": report[label]["f1-score"],
                "Support": report[label]["support"],
                "Accuracy": acc,
                "Primary_Macro_F1": macro_f1,
                "InvalidOutputs": invalid_outputs
            })

        # Macro avg
        rows.append({
            "Model": model_name,
            "Prompt": p,
            "PromptType": ["zero-shot", "one-shot", "few-shot"][p-1],
            "Label": "macro avg",
            "Precision": report["macro avg"]["precision"],
            "Recall": report["macro avg"]["recall"],
            "F1": macro_f1,
            "Support": report["macro avg"]["support"],
            "Accuracy": acc,
            "Primary_Macro_F1": macro_f1,
            "InvalidOutputs": invalid_outputs
        })

        # Weighted avg
        rows.append({
            "Model": model_name,
            "Prompt": p,
            "PromptType": ["zero-shot", "one-shot", "few-shot"][p-1],
            "Label": "weighted avg",
            "Precision": report["weighted avg"]["precision"],
            "Recall": report["weighted avg"]["recall"],
            "F1": report["weighted avg"]["f1-score"],
            "Support": report["weighted avg"]["support"],
            "Accuracy": acc,
            "Primary_Macro_F1": macro_f1,
            "InvalidOutputs": invalid_outputs
        })

        # Guardar prompt usado (referencia metodológica)
        with open(f"used_prompts/multiclass_{model_name}_prompt{p}.txt", "w", encoding="utf-8") as f:
            f.write(f"MODEL: {model_name}\nPROMPT VERSION: {p}\n")


# =========================
# Guardar CSV final
# =========================
df_multi_metrics = pd.DataFrame(rows)
df_multi_metrics.to_csv(
    "Mtemp.csv",
    index=False
)

print("\n✅ Saved: Mtemp.csv")


===== 🎨 MULTICLASS CLASSIFICATION (ALL MODELS) =====

MODEL: LLaMA-8B

--- Prompt 1 ---
Accuracy: 49.50%
Macro F1 (PRIMARY): 0.4796
                  precision    recall  f1-score   support

Generalized Hope       0.43      0.73      0.54        75
  Realistic Hope       0.65      0.32      0.43        68
Unrealistic Hope       0.59      0.39      0.47        57

        accuracy                           0.49       200
       macro avg       0.56      0.48      0.48       200
    weighted avg       0.55      0.49      0.48       200


--- Prompt 2 ---
Accuracy: 47.00%
Macro F1 (PRIMARY): 0.4561
                  precision    recall  f1-score   support

Generalized Hope       0.40      0.69      0.51        75
  Realistic Hope       0.62      0.31      0.41        68
Unrealistic Hope       0.57      0.37      0.45        57

        accuracy                           0.47       200
       macro avg       0.53      0.46      0.46       200
    weighted avg       0.52      0.47      0.46

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import os

os.makedirs("confusion_matrices", exist_ok=True)

for model_name in MODELS.keys():
    for p in [1, 2, 3]:
        col = f"{model_name}_p{p}"
        if col not in finMulti.columns:
            continue

        y_true = finMulti["multiclass"]
        y_pred = finMulti[col].apply(normalize_multiclass)

        labels = [
            "Generalized Hope",
            "Realistic Hope",
            "Unrealistic Hope"
        ]

        cm = confusion_matrix(y_true, y_pred, labels=labels)
        disp = ConfusionMatrixDisplay(cm, display_labels=labels)

        plt.figure(figsize=(6, 6))
        disp.plot(cmap="Blues", values_format="d")
        plt.title(f"{model_name} — Prompt {p}")
        plt.savefig(f"confusion_matrices/{model_name}_prompt{p}.png")
        plt.close()
